In [73]:
from collections import Counter
from enum import Enum

In [74]:
class NodeType(Enum):
    ALPHABETIC = 0
    TERMINAL = 1

class Node:
    level = None
    def __init__(self, left, right, weight, value, type : NodeType):
        self.left = left
        self.right = right
        self.weight = weight
        self.value = value
        self.type = type

#### Combining

In [75]:
def find_min_adjacent(nodes_list, index):
    min_adj = nodes_list[index].weight + nodes_list[index + 1].weight
    min_adj_index = index + 1
    for i in range(index + 1, len(nodes_list)):
        if nodes_list[i].weight + nodes_list[index].weight < min_adj:
                min_adj = nodes_list[i].weight + nodes_list[index].weight
                min_adj_index = i
        if nodes_list[i].type is NodeType.ALPHABETIC:
            break
    return min_adj_index

In [76]:
# Todo fix!
def combine(nodes_list):
    while len(nodes_list) > 1:
        min_sum = nodes_list[0].weight + nodes_list[1].weight
        min_left_index = 0
        min_right_index = 1
        for i in range(0, len(nodes_list) - 1):
            min_adj_index = find_min_adjacent(nodes_list, i)
            print("Min adj for", i, ":", min_adj_index)
            if nodes_list[i].weight + nodes_list[min_adj_index].weight < min_sum:
                min_sum = nodes_list[i].weight + nodes_list[min_adj_index].weight
                min_left_index = i
                min_right_index = min_adj_index
        print("United: ", nodes_list[min_left_index].weight, "+", nodes_list[min_right_index].weight, "(", min_left_index, "," , min_right_index, ")", nodes_list[min_left_index].value, nodes_list[min_right_index].value, min_sum)
        nodes_list[min_left_index] = Node(nodes_list[min_left_index], nodes_list[min_right_index], min_sum, None, NodeType.TERMINAL)
        nodes_list.pop(min_right_index)

#### Levels counting

In [77]:
def count_levels(node):
    if node.left != None :
        node.left.level = node.level + 1
        node.right.level = node.level + 1
        count_levels(node.left)
        count_levels(node.right)

#### Restructuring

In [78]:
def restrict(nodes_list):
    stack = list()
    i = 0
    new_node = None
    while i <= len(nodes_list):
        if len(stack) < 2 or stack[-1].level != stack[-2].level:
            if i == len(nodes_list):
                break
            stack.append(nodes_list[i])
            i+=1
        else:
            left = stack.pop()
            right = stack.pop()
            new_node = Node(left, right, None, None, NodeType.TERMINAL)
            new_node.level = left.level - 1
            stack.append(new_node)
        print("Stack:")
        for item in stack:
            print(item.level, ",")
    return new_node

#### Generate code table

In [79]:
def generate_code_table(root_node : Node, table_dict, current_bits):
    if root_node.left is not None and root_node.right is not None:
        left_bits = current_bits.copy()
        right_bits = current_bits.copy()
        left_bits.append(0)
        right_bits.append(1)
        print("right: ", right_bits, " left:", left_bits)
        generate_code_table(root_node.left, table_dict, left_bits)
        generate_code_table(root_node.right, table_dict, right_bits)
    else:
        table_dict[root_node.value] = current_bits

#### For tests

In [80]:
def output_tree(root : Node):
    stack = [root]
    while len(stack) > 0 :
        item = stack.pop(0)
        print(item.value, ":", item.level)
        if item.left is not None:
            stack.append(item.left)
            stack.append(item.right)

### Main body

In [81]:
input_file = open("../test/test.txt", "r")
symbols_weights = Counter(input_file.read())
sorted(symbols_weights.items())

[('a', 8),
 ('b', 6),
 ('c', 2),
 ('d', 3),
 ('e', 4),
 ('f', 7),
 ('g', 11),
 ('h', 9),
 ('i', 8),
 ('j', 1),
 ('k', 3)]

In [82]:
nodes_list = list(map(lambda symbol_weight: Node(None, None, symbol_weight[1], symbol_weight[0], NodeType.ALPHABETIC), sorted(symbols_weights.items())))
leaves = nodes_list.copy()
for node in nodes_list:
    print(node.value, sep=" ")

a
b
c
d
e
f
g
h
i
j
k


In [83]:
combine(nodes_list)
root = nodes_list[0]

Min adj for 0 : 1
Min adj for 1 : 2
Min adj for 2 : 3
Min adj for 3 : 4
Min adj for 4 : 5
Min adj for 5 : 6
Min adj for 6 : 7
Min adj for 7 : 8
Min adj for 8 : 9
Min adj for 9 : 10
United:  1 + 3 ( 9 , 10 ) j k 4
Min adj for 0 : 1
Min adj for 1 : 2
Min adj for 2 : 3
Min adj for 3 : 4
Min adj for 4 : 5
Min adj for 5 : 6
Min adj for 6 : 7
Min adj for 7 : 8
Min adj for 8 : 9
United:  2 + 3 ( 2 , 3 ) c d 5
Min adj for 0 : 1
Min adj for 1 : 3
Min adj for 2 : 3
Min adj for 3 : 4
Min adj for 4 : 5
Min adj for 5 : 6
Min adj for 6 : 7
Min adj for 7 : 8
United:  5 + 4 ( 2 , 3 ) None e 9
Min adj for 0 : 1
Min adj for 1 : 3
Min adj for 2 : 3
Min adj for 3 : 4
Min adj for 4 : 5
Min adj for 5 : 6
Min adj for 6 : 7
United:  8 + 4 ( 6 , 7 ) i None 12
Min adj for 0 : 1
Min adj for 1 : 3
Min adj for 2 : 3
Min adj for 3 : 4
Min adj for 4 : 5
Min adj for 5 : 6
United:  6 + 7 ( 1 , 3 ) b f 13
Min adj for 0 : 2
Min adj for 1 : 2
Min adj for 2 : 3
Min adj for 3 : 4
Min adj for 4 : 5
United:  8 + 9 ( 0 , 2 ) 

In [84]:
root.level = 0
count_levels(root)
output_tree(root)

None : 0
None : 1
None : 1
None : 2
None : 2
None : 2
None : 2
a : 3
None : 3
g : 3
h : 3
b : 3
f : 3
i : 3
None : 3
None : 4
e : 4
j : 4
k : 4
c : 5
d : 5


In [85]:
root = restrict(leaves)

Stack:
3 ,
Stack:
3 ,
3 ,
Stack:
2 ,
Stack:
2 ,
5 ,
Stack:
2 ,
5 ,
5 ,
Stack:
2 ,
4 ,
Stack:
2 ,
4 ,
4 ,
Stack:
2 ,
3 ,
Stack:
2 ,
3 ,
3 ,
Stack:
2 ,
2 ,
Stack:
1 ,
Stack:
1 ,
3 ,
Stack:
1 ,
3 ,
3 ,
Stack:
1 ,
2 ,
Stack:
1 ,
2 ,
3 ,
Stack:
1 ,
2 ,
3 ,
4 ,
Stack:
1 ,
2 ,
3 ,
4 ,
4 ,
Stack:
1 ,
2 ,
3 ,
3 ,
Stack:
1 ,
2 ,
2 ,
Stack:
1 ,
1 ,
Stack:
0 ,


In [86]:
code_table = {}
generate_code_table(root, code_table, [])
code_table


right:  [1]  left: [0]
right:  [0, 1]  left: [0, 0]
right:  [0, 0, 1]  left: [0, 0, 0]
right:  [0, 0, 0, 1]  left: [0, 0, 0, 0]
right:  [0, 1, 1]  left: [0, 1, 0]
right:  [1, 1]  left: [1, 0]
right:  [1, 0, 1]  left: [1, 0, 0]
right:  [1, 0, 1, 1]  left: [1, 0, 1, 0]
right:  [1, 0, 1, 1, 1]  left: [1, 0, 1, 1, 0]
right:  [1, 1, 1]  left: [1, 1, 0]


{'k': [0, 0, 0, 0],
 'j': [0, 0, 0, 1],
 'i': [0, 0, 1],
 'h': [0, 1, 0],
 'g': [0, 1, 1],
 'f': [1, 0, 0],
 'e': [1, 0, 1, 0],
 'd': [1, 0, 1, 1, 0],
 'c': [1, 0, 1, 1, 1],
 'b': [1, 1, 0],
 'a': [1, 1, 1]}